<a href="https://colab.research.google.com/github/linyuehzzz/hedetniemi_distance/blob/master/hedetniemi_distance.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##**Hedetniemi Matrix Sum**
This code is used to implement the [Hedetniemi Matrix Sum](https://deepblue.lib.umich.edu/handle/2027.42/59763).  
Yue Lin (lin.3326 at osu.edu)  
Created: 5/6/2020

#### **Generate graph data** 

In [0]:
## [node i, node j, distance between node i and j]
## using data from example 1: San Francisco Bay Area Graph of Time-Distances (in minutes)
data = [[1, 2, 30], [1, 4, 30], [1, 9, 40],
        [2, 3, 25], [2, 4, 40], [3, 4, 50],
        [4, 5, 30], [4, 6, 20], [5, 7, 25],
        [6, 7, 20], [6, 9, 20], [7, 8, 25],
        [8, 9,20]]

#### **Implementation 1: list** 

Construct distance matrix

In [23]:
from timeit import default_timer

def distance_matrix(graph):
  ## calculate number of nodes
  n = max([g[1] for g in graph])

  ## calculate distance matrix
  INF = float('inf')
  dist_mtx = [[INF] * n for i in range(n)]
  for g in graph:
    i = g[0] - 1
    j = g[1] - 1
    d = g[2]
    dist_mtx[i][j] = d
    dist_mtx[j][i] = d

  ## set diagonal to 0
  for i in range(n):
    dist_mtx[i][i] = 0
 
  return dist_mtx, n


## print time costs
start = default_timer()
dist_mtx, n = distance_matrix(data)
stop = default_timer()
print('Time: ', stop - start)

## print distance matrix
print("Distance matrix: ")
for line in dist_mtx:
  print(line)

Time:  4.834100036532618e-05
Distance matrix: 
[0, 30, inf, 30, inf, inf, inf, inf, 40]
[30, 0, 25, 40, inf, inf, inf, inf, inf]
[inf, 25, 0, 50, inf, inf, inf, inf, inf]
[30, 40, 50, 0, 30, 20, inf, inf, inf]
[inf, inf, inf, 30, 0, inf, 25, inf, inf]
[inf, inf, inf, 20, inf, 0, 20, inf, 20]
[inf, inf, inf, inf, 25, 20, 0, 25, inf]
[inf, inf, inf, inf, inf, inf, 25, 0, 20]
[40, inf, inf, inf, inf, 20, inf, 20, 0]


Calculate Hedetniemi Matrix Sum

In [26]:
from timeit import default_timer

def hede_distance(matrix, n):
  mtx_a_t = [[INF] * n for i in range(n)]
  mtx_a_t_1 = matrix

  p = True
  while p:
    for i in range(n):
      a = mtx_a_t_1[i]
      for j in range(n):
        b = [row[j] for row in matrix]
        mtx_a_t[i][j] = min([a[k] + b[k] for k in range(n)])
    
    if mtx_a_t == mtx_a_t_1:
      p =  False
    else:
      mtx_a_t_1 = mtx_a_t   
  
  return mtx_a_t


## print time costs
start = default_timer()
mtx_a_t = hede_distance(dist_mtx, n)
stop = default_timer()
print('Time: ', stop - start)

## print shortest path matrix
print("Shortest path matrix: ")
for line in mtx_a_t:
  print(line)

Time:  0.0004263869996066205
Shortest path matrix: 
[0, 30, 55, 30, 60, 50, 70, 60, 40]
[30, 0, 25, 40, 70, 60, 80, 90, 70]
[55, 25, 0, 50, 80, 70, 90, 115, 90]
[30, 40, 50, 0, 30, 20, 40, 60, 40]
[60, 70, 80, 30, 0, 45, 25, 50, 65]
[50, 60, 70, 20, 45, 0, 20, 40, 20]
[70, 80, 90, 40, 25, 20, 0, 25, 40]
[60, 90, 115, 60, 50, 40, 25, 0, 20]
[40, 70, 90, 40, 65, 20, 40, 20, 0]


#### **Implementation 2: numpy** 

Construct distance matrix

In [33]:
from timeit import default_timer
import numpy as np

def distance_matrix(graph):
  ## calculate number of nodes
  n = np.amax(graph[:,1])

  ## calculate distance matrix
  dist_mtx = np.full((n,n), np.inf)
  for g in graph:
    i = g[0] - 1
    j = g[1] - 1
    d = g[2]
    dist_mtx[i,j] = d
    dist_mtx[j,i] = d

  ## set diagonal to 0
  for i in range(n):
    dist_mtx[i,i] = 0
 
  return dist_mtx, n


## print time costs
start = default_timer()
dist_mtx, n = distance_matrix(np.array(data))
stop = default_timer()
print('Time: ', stop - start)

## print distance matrix
print("Distance matrix: ")
for line in dist_mtx:
  print(line)

Time:  0.0010771590023068711
Distance matrix: 
[ 0. 30. inf 30. inf inf inf inf 40.]
[30.  0. 25. 40. inf inf inf inf inf]
[inf 25.  0. 50. inf inf inf inf inf]
[30. 40. 50.  0. 30. 20. inf inf inf]
[inf inf inf 30.  0. inf 25. inf inf]
[inf inf inf 20. inf  0. 20. inf 20.]
[inf inf inf inf 25. 20.  0. 25. inf]
[inf inf inf inf inf inf 25.  0. 20.]
[40. inf inf inf inf 20. inf 20.  0.]


Calculate Hedetniemi Matrix Sum

In [34]:
from timeit import default_timer
import numpy as np

def hede_distance(matrix, n):
  mtx_a_t = np.full((n,n), np.inf)
  mtx_a_t_1 = matrix

  p = True
  while p:
    for i in range(n):
      a = mtx_a_t_1[i]
      for j in range(n):
        b = matrix[:,j]
        mtx_a_t[i,j] = min([a[k] + b[k] for k in range(n)])
    
    if np.array_equal(mtx_a_t, mtx_a_t_1):
      p =  False
    else:
      mtx_a_t_1 = mtx_a_t   
  
  return mtx_a_t


## print time costs
start = default_timer()
mtx_a_t = hede_distance(dist_mtx, n)
stop = default_timer()
print('Time: ', stop - start)

## print shortest path matrix
print("Shortest path matrix: ")
for line in mtx_a_t:
  print(line)

Time:  0.0010203220008406788
Shortest path matrix: 
[ 0. 30. 55. 30. 60. 50. 70. 60. 40.]
[30.  0. 25. 40. 70. 60. 80. 90. 70.]
[ 55.  25.   0.  50.  80.  70.  90. 115.  90.]
[30. 40. 50.  0. 30. 20. 40. 60. 40.]
[60. 70. 80. 30.  0. 45. 25. 50. 65.]
[50. 60. 70. 20. 45.  0. 20. 40. 20.]
[70. 80. 90. 40. 25. 20.  0. 25. 40.]
[ 60.  90. 115.  60.  50.  40.  25.   0.  20.]
[40. 70. 90. 40. 65. 20. 40. 20.  0.]


#### **Implementation 3: tensorflow** 

In [32]:
!pip install -q tf-nightly

     |████████████████████████████████| 520.9MB 29kB/s 
     |████████████████████████████████| 2.9MB 50.6MB/s 
     |████████████████████████████████| 460kB 40.5MB/s 


Construct distance matrix

In [35]:
from timeit import default_timer
import tensorflow as tf
import numpy as np

def distance_matrix(graph):
  ## calculate number of nodes
  n = tf.reduce_max(graph[:,1])

  ## calculate distance matrix
  dist_mtx = tf.fill([n, n], np.inf)
  dist_mtx = map_fn(lambda x: x[0] * x[1], graph)
  for g in graph:
    i = g[0] - 1
    j = g[1] - 1
    d = g[2]
    dist_mtx[i,j] = d
    dist_mtx[j,i] = d

  ## set diagonal to 0
  tf.matrix_set_diag(dist_mtx, tf.zeros(dist_mtx.shape[0:-1]), name=None)
 
  return dist_mtx, n


## print time costs
start = default_timer()
dist_mtx, n = distance_matrix(tf.constant(data))
stop = default_timer()
print('Time: ', stop - start)

## print distance matrix
print("Distance matrix: ")
for line in dist_mtx:
  print(line)

TypeError: ignored